In [1]:
from espuma import Case_Directory
import pyvista as pv

In [2]:
of_case = Case_Directory("./example_my_cavity")
of_case

Case_Directory(/home/edsaa/Repositories/foamy/examples/example_my_cavity)

In [3]:
reader = of_case.get_vtk_reader()

In [4]:
# Using pyvista
print(f"Available Time Values: {reader.time_values}")

# Using espuma
print(f"Available Time Values: {of_case.list_times}")

Available Time Values: [0.0, 0.1, 0.2, 0.3, 0.4, 0.5]
Available Time Values: [0.0, 0.1, 0.2, 0.3, 0.4, 0.5]


In [5]:
reader.set_active_time_value(0.5)
reader.cell_to_point_creation = True

In [6]:
mesh = reader.read()
mesh

MultiBlock (0x7ff3e9571b40)
  N Blocks    2
  X Bounds    0.000, 0.100
  Y Bounds    0.000, 0.100
  Z Bounds    0.000, 0.010

In [7]:
internal_mesh = mesh["internalMesh"]
boundaries = mesh["boundary"]

In [8]:
internal_mesh

UnstructuredGrid (0x7ff3e95719c0)
  N Cells:    400
  N Points:   882
  X Bounds:   0.000e+00, 1.000e-01
  Y Bounds:   0.000e+00, 1.000e-01
  Z Bounds:   0.000e+00, 1.000e-02
  N Arrays:   4

In [9]:
boundaries

MultiBlock (0x7ff3e9571c00)
  N Blocks    3
  X Bounds    0.000, 0.100
  Y Bounds    0.000, 0.100
  Z Bounds    0.000, 0.010

In [10]:
def slice_z_center(mesh):
    """Slice mesh through center in z normal direction, move to z=0."""
    slice_mesh = mesh.slice(normal='z')
    slice_mesh.translate((0, 0, -slice_mesh.center[-1]), inplace=True)
    return slice_mesh

def plot_sliced_pressure(mesh):
    plotting_options = dict(
        scalars="p", 
        cmap='bwr', 
        ambient=0.8,
        show_edges=False
    ) 

    internal_mesh = mesh["internalMesh"]
    boundaries = mesh["boundary"]

    slice_internal_mesh = slice_z_center(internal_mesh)
    # slice_boundaries = pv.MultiBlock(
    #     {key: slice_z_center(boundaries[key]) for key in boundaries.keys()}
    # )

    plotter = pv.Plotter()
    plotter.add_mesh(slice_internal_mesh,**plotting_options)
    plotter.view_xy()
    plotter.enable_parallel_projection()
    plotter.show_bounds(use_3d_text=False)
    plotter.show()

In [11]:
plot_sliced_pressure(mesh)

Widget(value='<iframe src="http://localhost:45117/index.html?ui=P_0x7ff43e6956d0_0&reconnect=auto" class="pyvi…

In [12]:
of_denser = Case_Directory.clone_from_template(
    of_case,
    "example_denser",
    overwrite=True
)

In [13]:
of_case.constant.transportProperties["nu"]

'[ 0 2 -1 0 0 0 0 ] 0.01'

In [14]:
of_denser.constant.transportProperties["nu"] = '[ 0 2 -1 0 0 0 0 ] 0.1'

In [15]:
of_denser.constant.transportProperties

Dict_File(/home/edsaa/Repositories/foamy/examples/example_denser/constant/transportProperties)

In [16]:
of_denser._blockMesh()
of_denser._runCase()

blockMesh finished successfully!
icoFoam finished successfully!


In [17]:
reader = of_denser.get_vtk_reader()
reader.set_active_time_value(0.5)
reader.cell_to_point_creation = True
mesh = reader.read()

In [18]:
plot_sliced_pressure(mesh)

Widget(value='<iframe src="http://localhost:45117/index.html?ui=P_0x7ff3e3f6a9d0_1&reconnect=auto" class="pyvi…